# Replication of paper "Econometrics at Scale - Spark Up Big Data in Economics", Chapter 4.4 (B. Bluhm and J. Cutura)
This notebook implements a distributed time series forecasting exercise in PySpark as described in more detail in the paper "Econometrics at Scale - Spark Up Big Data in Economics" by Benjamin Bluhm and Jannic Cutura.

## Create dictionary with config parameters

In [9]:
import os

# initialize dictionary with config parameters
config = {}

# Define working directory
wdir = 's3://my-bucket/'

# Define AWS S3 endpoint for your region
config['s3_host'] = 's3.eu-central-1.amazonaws.com'

# Define Path to store results
config['path_training_data_csv'] = os.path.join(wdir, 'data/time_series/rawdata_small.csv')
config['path_training_data_parquet'] = os.path.join(wdir, 'output/time_series/parquet/')
config['path_forecasts'] = os.path.join(wdir, 'output/time_series/forecasts/')
config['path_models'] = os.path.join(wdir, 'output/time_series/ models/')
config['path_code'] = os.path.join(wdir, 'scripts/time_ series/')

# Define series and evaluation lengths
config['len_series'] = 1000
config['len_eval'] = 50

## Install necessary Python packages on all cluster nodes

In [ ]:
sc.install_pypi_package("s3fs")
sc.install_pypi_package("joblib")
sc.install_pypi_package("s3io")
sc.install_pypi_package("pandas")
sc.install_pypi_package("fastparquet")
sc.install_pypi_package("statsmodels")

## Partition and save dataset to Parquet format
In this step, we load the sample csv file into a Spark DataFrame, repartition the DataFrame by 'ID' column and save the data in Parquet file format

In [5]:
df = spark.read.csv(config['path_training_data_csv'],header = True, inferSchema=True)

df.repartition("ID").write.option("compression", "gzip").mode("overwrite")\
    .partitionBy("ID").parquet(config['path_training_data_parquet'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Add Python module to Spark context for distributed execution

In [6]:
spark.sparkContext.addPyFile(config['path_code'] + '/fit_model_and_forecast.py')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Perform model fitting and forecasting (distributed execution scheme)
In this step, we run the forecasting exercise in a distributed fashion

In [7]:
# Load time series data into Spark dataframe
df = spark.read.parquet(config['path_training_data_parquet'])

# Create RDD with dictinct IDs and repartition dataframe into 100 chunks
time_series_ids = df.select("ID").distinct().repartition(100).rdd

# Function to import Python module on Spark executor for parallel forecasting
def import_module_on_spark_executor(time_series_ids, config):
    from fit_model_and_forecast import fit_model_and_forecast
    return fit_model_and_forecast(time_series_ids, config, cloud=True)

# Parallel model fitting and forecasting
time_series_ids.foreach(lambda x: import_module_on_spark_executor(x, config))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…